In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','w0','w1','w2','w3','w4','w5','w6','day','month','year', 'accuracy']
for i in range(0,24):
    feature_list.append('h' + str(i))

In [3]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    #df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    for i in range(0,24):
        df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24) * fw[2]
    
    df['w0'] = ((d_times.weekday + 0) % 7) * fw[3]
    df['w1'] = ((d_times.weekday + 1) % 7) * fw[3]
    df['w2'] = ((d_times.weekday + 2) % 7) * fw[3]
    df['w3'] = ((d_times.weekday + 3) % 7) * fw[3]
    df['w4'] = ((d_times.weekday + 4) % 7) * fw[3]
    df['w5'] = ((d_times.weekday + 5) % 7) * fw[3]
    df['w6'] = ((d_times.weekday + 6) % 7) * fw[3]
    
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    #df.accuracy = df.accuracy.values * fw[7]
    df['accuracy'] = np.log10(df.accuracy) * fw[7]
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30)) * fw[8]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def calculate_distance(distances):
    return distances ** -2

In [9]:
def process_one_cell(df_train, df_test, th):    
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    row_ids = df_test.row_id
    
    best_k=np.floor(np.sqrt(len(df_train.index)/4*best_mul)/5)
    #best_k += best_k_offsite
    print('k:', best_k, flush=True)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=best_k.astype(int), weights=calculate_distance, 
                               metric='manhattan')
    clf.fit(df_train[feature_list], df_train.place_id)
    predictions = clf.predict_proba(df_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
def run_prediction(df_train, df_test):
    df_train = prepare_data(df_train)
    df_test = prepare_data(df_test)
    df_test['p1'] = np.nan
    df_test['p2'] = np.nan
    df_test['p3'] = np.nan
    
    prediction_result = process_one_cell(df_train, df_test, 1)
    prediction_result.sort_index(inplace=True)
    return prediction_result

In [7]:
def run_validation():
    df = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.time <= 786239 * 0.875) & \
                               (df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_validation_cell = df[(df.time > 786239 * 0.875) & \
                                    (df.x >= min_x) & \
                                    (df.x < max_x) & \
                                    (df.y >= min_y) & \
                                    (df.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_validation_cell)
            
            # Calculate score
            prediction_result.sort_index(inplace=True)
            prediction_result['score'] = (prediction_result.p1 == df_validation_cell.place_id) * 1
            prediction_result['score'] += (prediction_result.p2 == df_validation_cell.place_id) * 0.5
            prediction_result['score'] += (prediction_result.p3 == df_validation_cell.place_id) * 0.33

            score = prediction_result.score.mean()
            #print('s1:', score, flush=True)
            total_score += score
            score_count += 1
            
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
    print("Final:", total_score/score_count, flush=True)


In [8]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 20, 4.5]

best_k_offsite = -4
run_validation()

best_k_offsite = -2
run_validation()

best_k_offsite = 0
run_validation()

best_k_offsite = 2
run_validation()

best_k_offsite = 4
run_validation()

	Line 211: Final: 0.5479812517927083
	Line 422: Final: 0.547873847316114
	Line 633: Final: 0.5476198299889753
	Line 844: Final: 0.5474168240104926
	Line 1055: Final: 0.5471965125056125

k: 35.0
k: 38.0
k: 39.0
k: 40.0
k: 38.0
k: 37.0
k: 38.0
k: 38.0
k: 39.0
k: 38.0
k: 39.0
k: 39.0
k: 37.0
k: 36.0
k: 37.0
k: 36.0
k: 38.0
k: 38.0
k: 39.0
k: 36.0
Elapsed time overall: 538.8613576889038 seconds 0
k: 37.0
k: 41.0
k: 41.0
k: 40.0
k: 41.0
k: 41.0
k: 40.0
k: 40.0
k: 40.0
k: 41.0
k: 42.0
k: 39.0
k: 39.0
k: 41.0
k: 40.0
k: 43.0
k: 41.0
k: 40.0
k: 41.0
k: 37.0
Elapsed time overall: 613.6858987808228 seconds 1
k: 38.0
k: 44.0
k: 43.0
k: 40.0
k: 41.0
k: 43.0
k: 40.0
k: 41.0
k: 43.0
k: 42.0
k: 41.0
k: 42.0
k: 40.0
k: 42.0
k: 41.0
k: 42.0
k: 42.0
k: 43.0
k: 43.0
k: 39.0
Elapsed time overall: 590.3099620342255 seconds 2
k: 36.0
k: 40.0
k: 42.0
k: 42.0
k: 41.0
k: 42.0
k: 42.0
k: 41.0
k: 42.0
k: 41.0
k: 42.0
k: 40.0
k: 40.0
k: 41.0
k: 40.0
k: 42.0
k: 42.0
k: 42.0
k: 41.0
k: 37.0
Elapsed time overall: 524.5748240947723 seconds 3
k: 37.0
k: 42.0
k: 42.0
k: 41.0
k: 41.0
k: 42.0
k: 42.0
k: 42.0
k: 42.0
k: 41.0
k: 42.0
k: 42.0
k: 42.0
k: 41.0
k: 41.0
k: 41.0
k: 42.0
k: 42.0
k: 41.0
k: 38.0


In [10]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 20, 4.5]

best_mul = 0.8
run_validation()

best_mul = 0.9
run_validation()

best_mul = 1.0
run_validation()

best_mul = 1.1
run_validation()

best_mul = 1.2
run_validation()

	Line 211: Final: 0.5480491884069056
	Line 422: Final: 0.5478869007890222
	Line 633: Final: 0.5476198299889753
	Line 844: Final: 0.5473946392742225
	Line 1055: Final: 0.5471685033887294

k: 35.0
k: 37.0
k: 39.0
k: 39.0
k: 38.0
k: 36.0
k: 38.0
k: 38.0
k: 39.0
k: 38.0
k: 38.0
k: 39.0
k: 36.0
k: 36.0
k: 37.0
k: 36.0
k: 37.0
k: 38.0
k: 38.0
k: 36.0
Elapsed time overall: 412.70688819885254 seconds 0
k: 36.0
k: 40.0
k: 40.0
k: 39.0
k: 40.0
k: 40.0
k: 39.0
k: 40.0
k: 40.0
k: 40.0
k: 41.0
k: 39.0
k: 38.0
k: 40.0
k: 40.0
k: 42.0
k: 40.0
k: 39.0
k: 41.0
k: 37.0
Elapsed time overall: 448.6332859992981 seconds 1
k: 38.0
k: 42.0
k: 42.0
k: 39.0
k: 40.0
k: 42.0
k: 40.0
k: 41.0
k: 42.0
k: 41.0
k: 41.0
k: 41.0
k: 39.0
k: 41.0
k: 40.0
k: 41.0
k: 41.0
k: 42.0
k: 42.0
k: 38.0
Elapsed time overall: 486.01372504234314 seconds 2
k: 36.0
k: 39.0
k: 41.0
k: 41.0
k: 40.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 39.0
k: 39.0
k: 40.0
k: 40.0
k: 41.0
k: 41.0
k: 41.0
k: 40.0
k: 37.0
Elapsed time overall: 462.83696126937866 seconds 3
k: 37.0
k: 41.0
k: 41.0
k: 40.0
k: 40.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 40.0
k: 40.0
k: 41.0
k: 41.0
k: 41.0
k: 41.0
k: 37

In [11]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 23, 4.5]


best_mul = 0.6
run_validation()

best_mul = 0.7
run_validation()

best_mul = 0.8
run_validation()

best_mul = 0.9
run_validation()

	Line 211: Final: 0.5483083948027339
	Line 422: Final: 0.5482438867368357
	Line 633: Final: 0.5481256287023172
	Line 844: Final: 0.5479414877903609

k: 30.0
k: 32.0
k: 34.0
k: 34.0
k: 33.0
k: 32.0
k: 33.0
k: 33.0
k: 34.0
k: 33.0
k: 33.0
k: 33.0
k: 31.0
k: 31.0
k: 32.0
k: 31.0
k: 32.0
k: 33.0
k: 33.0
k: 31.0
Elapsed time overall: 369.0889766216278 seconds 0
k: 31.0
k: 34.0
k: 35.0
k: 34.0
k: 34.0
k: 35.0
k: 34.0
k: 34.0
k: 34.0
k: 35.0
k: 35.0
k: 33.0
k: 33.0
k: 34.0
k: 34.0
k: 36.0
k: 35.0
k: 34.0
k: 35.0
k: 32.0
Elapsed time overall: 404.1032383441925 seconds 1
k: 33.0
k: 37.0
k: 36.0
k: 34.0
k: 35.0
k: 36.0
k: 34.0
k: 35.0
k: 36.0
k: 35.0
k: 35.0
k: 35.0
k: 34.0
k: 35.0
k: 35.0
k: 36.0
k: 35.0
k: 36.0
k: 36.0
k: 33.0
Elapsed time overall: 429.87149715423584 seconds 2
k: 31.0
k: 34.0
k: 35.0
k: 35.0
k: 35.0
k: 36.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 34.0
k: 34.0
k: 34.0
k: 34.0
k: 36.0
k: 36.0
k: 36.0
k: 34.0
k: 32.0
Elapsed time overall: 407.3577151298523 seconds 3
k: 32.0
k: 36.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 36.0
k: 36.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 35.0
k: 36.0
k: 35.0
k: 32.0

In [12]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 23, 4.5]


best_mul = 0.2
run_validation()

best_mul = 0.3
run_validation()

best_mul = 0.4
run_validation()

best_mul = 0.43
run_validation()

best_mul = 0.45
run_validation()

best_mul = 0.47
run_validation()

best_mul = 0.5
run_validation()

	Line 211: Final: 0.5450306408938852
	Line 422: Final: 0.547089038872003
	Line 633: Final: 0.5478461541064292
	Line 844: Final: 0.5479758470546281
	Line 1055: Final: 0.5480835586301165
	Line 1266: Final: 0.5481300977336594
	Line 1477: Final: 0.5482278262311948

k: 17.0
k: 18.0
k: 19.0
k: 19.0
k: 19.0
k: 18.0
k: 19.0
k: 19.0
k: 19.0
k: 19.0
k: 19.0
k: 19.0
k: 18.0
k: 18.0
k: 18.0
k: 18.0
k: 18.0
k: 19.0
k: 19.0
k: 18.0
Elapsed time overall: 285.3513751029968 seconds 0
k: 18.0
k: 20.0
k: 20.0
k: 19.0
k: 20.0
k: 20.0
k: 19.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 19.0
k: 19.0
k: 20.0
k: 20.0
k: 21.0
k: 20.0
k: 19.0
k: 20.0
k: 18.0
Elapsed time overall: 315.8986999988556 seconds 1
k: 19.0
k: 21.0
k: 21.0
k: 19.0
k: 20.0
k: 21.0
k: 20.0
k: 20.0
k: 21.0
k: 20.0
k: 20.0
k: 20.0
k: 19.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 21.0
k: 21.0
k: 19.0
Elapsed time overall: 347.555104970932 seconds 2
k: 18.0
k: 19.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 19.0
k: 19.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 18.0
Elapsed time overall: 323.8220007419586 seconds 3
k: 18.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 20.0
k: 18.0
E

In [ ]:
def run_test():
    
    # Run test
    df = pd.read_csv('../../train.csv',
                           usecols=['row_id','x','y','accuracy','time','place_id'])
    df_test = pd.read_csv('../../test.csv',
                           usecols=['row_id','x','y','accuracy','time'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    total_result = pd.DataFrame()
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_test_cell = df_test[(df_test.x >= min_x) & \
                                   (df_test.x < max_x) & \
                                   (df_test.y >= min_y) & \
                                   (df_test.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_test_cell)
            total_result = total_result.append(prediction_result)
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
            
    
    total_result.sort_index(inplace=True)
    total_result['place_id'] = total_result.p1.astype(str) + " " + \
                               total_result.p2.astype(str) + " " + \
                               total_result.p3.astype(str)
    total_result[['row_id', 'place_id']].to_csv('Baseline620.csv', index=False)

In [ ]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 16, 4.5]
run_test()

In [ ]:
0.58065